# Analysis

In [484]:
%reload_ext autoreload
%autoreload 2

In [485]:
#export
from exp.nb_00 import *
from exp.nb_01 import *
from exp.nb_02 import *
import os
import shutil
import pickle
import re
import subprocess
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
import numpy as np
from climatools.lblnew.export import vector_to_F77

In [487]:
param = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})
param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': '79f2292', 'band': 9, 'molecule': 'h2o', 'atmpro': 'saw', 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(30, 250), (300, 250)], 'ng_refs': [3, 9], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)], 'cosz': 1, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [488]:
PATH_FORTRAN = Path('/chia_cluster/home/jackyu/radiation/crdnew-sw/examples')

In [489]:
path = PATH_FORTRAN/'/'.join(sorted(f'{n}_{v}' for n, v in vars(param).items()))
runner = LBLnewBestfitSWRun(path, param)

In [490]:
runner.input_params()
proc = runner.build()
out, err = proc.communicate()

In [492]:
runner.run()

Job <435071> is submitted to queue <serial>.



In [494]:
! qstat -u jackyu

No matching job found
serial;  type=BATCH;  [ENABLED];  pri=35
3 run;   7 wait;


v4q;  type=BATCH;  [ENABLED];  pri=30
192 run;   0 wait;


v4qexp;  type=BATCH;  [ENABLED];  pri=30
144 run;   192 wait;


LCCR_Q;  type=BATCH;  [ENABLED];  pri=30
1152 run;   0 wait;


URBAN_Q;  type=BATCH;  [ENABLED];  pri=30
161 run;   448 wait;


orc;  type=BATCH;  [ENABLED];  pri=30
0 run;   0 wait;


RCEC_Q;  type=BATCH;  [ENABLED];  pri=30
96 run;   0 wait;


medium_priority;  type=BATCH;  [ENABLED];  pri=10
0 run;   0 wait;




## Set up analysis directory

In [495]:
PATH_ANALYSIS

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw')

In [496]:
# We are going to test directly at a location inside the 'analysis_-_new_kdist_param_sw' repository, 
# because we will try to git-add and git-commit the executed notebook.
PATH = PATH_ANALYSIS/'/'.join(sorted(f'{n}_{v}' for n, v in vars(param).items()))
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]')

In [497]:
PATH.mkdir(exist_ok=True, parents=True)

## Load the analysis notebook template

In [463]:
! pwd

/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev


In [350]:
os.chdir('/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/')

In [351]:
# For now, we assume that the analysis notebook template resides in the current directory
# in which we're developing things: /chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev/.
with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:
    analysis_nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

In [352]:
str(vars(runner.param))

"{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}"

In [353]:
analysis_nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(runner.param)})'''
analysis_nb['cells'][2]

{'cell_type': 'code',
 'execution_count': 3,
 'metadata': {},
 'outputs': [],
 'source': "PARAM = LBLnewBestfitSWParam(**{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0})"}

In [354]:
analysis_nb['cells'][3]['source'] = f'''PATH = Path("{runner.path}")'''
analysis_nb['cells'][3]

{'cell_type': 'code',
 'execution_count': 4,
 'metadata': {'collapsed': True},
 'outputs': [],
 'source': 'PATH = Path("test1/test1")'}

In [355]:
Path(runner.path)

PosixPath('test1/test1')

## Run the analysis notebook  
Useful: https://nbconvert.readthedocs.io/en/latest/execute_api.html

In [357]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test'),
 [])

In [358]:
ep = ExecutePreprocessor(timeout=600)
ep.preprocess(analysis_nb, {});

In [359]:
with open(PATH/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
    nbformat.write(analysis_nb, f)

In [360]:
PATH

PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test')

## Git add the executed notebook

In [361]:
PATH, list(PATH.iterdir())

(PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test'),
 [PosixPath('/chia_cluster/home/jackyu/radiation/analysis_-_new_kdist_param_sw/analysis_test/analysis_-_lblnew-bestfit-sw.ipynb')])

In order to add the executed notebook to the 'analysis_-_lblnew-bestfit-sw' directory, we need to be under that directory at least. 

In [363]:
cwd = os.getcwd()
cwd

'/chia_cluster/home/jackyu/climatools/climatools/notebooks/sw_dev'

In [364]:
os.chdir(PATH)

In [365]:
! ls

analysis_-_lblnew-bestfit-sw.ipynb


In [370]:
proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [371]:
out, err = proc.communicate()

In [372]:
print(out.decode(), err.decode())

In [373]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   analysis_-_lblnew-bestfit-sw.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../atmpro_None/band_9/commitnumber_None/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 6]/nv_10000/option_k_lookup_0/ref_pts_[(300, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]/analysis_-_lblnew-bestfit-sw.ipynb
	modified:   ../band09_-_h2o_atmpro_-_mls_-_cosz_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.ipynb_checkpoints/



## Git-commit the analysis notebook 
The commit message will contain the parameters for this particular case.

In [374]:
runner.param

<class 'exp.nb_00.LBLnewBestfitSWParam'>
{'dv': 0.001, 'nv': 10000, 'commitnumber': None, 'band': 9, 'molecule': 'h2o', 'atmpro': None, 'tsfc': None, 'vmin': 8200, 'vmax': 14290, 'conc': 'atmpro', 'ref_pts': [(300, 250), (300, 250)], 'ng_refs': [4, 6], 'ng_adju': [-3, 0], 'wgt': [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)], 'cosz': 1.0, 'rsfc': 0, 'klin': 3e-25, 'option_k_lookup': 0}

In [380]:
title = f"band{runner.param.band:02d} {runner.param.molecule} {runner.param.atmpro} cosz={runner.param.cosz} nf_refs={runner.param.ng_refs}"
title

'band09 h2o None cosz=1.0 nf_refs=[4, 6]'

In [388]:
body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(runner.param).items()))
print(body)

atmpro None
band 9
commitnumber None
conc atmpro
cosz 1.0
dv 0.001
klin 3e-25
molecule h2o
ng_adju [-3, 0]
ng_refs [4, 6]
nv 10000
option_k_lookup 0
ref_pts [(300, 250), (300, 250)]
rsfc 0
tsfc None
vmax 14290
vmin 8200
wgt [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]


In [391]:
proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True,
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [392]:
out, err = proc.communicate()

In [393]:
! git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../atmpro_None/band_9/commitnumber_None/conc_atmpro/cosz_1.0/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[4, 6]/nv_10000/option_k_lookup_0/ref_pts_[(300, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]/analysis_-_lblnew-bestfit-sw.ipynb
	modified:   ../band09_-_h2o_atmpro_-_mls_-_cosz_1.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../.ipynb_checkpoints/

no changes added to commit (use "git add" and/or "git commit -a")


In [394]:
! git log -1

commit 6fb47c1c797db1d3cc35dd5a3af515f290e4ef79 (HEAD -> master)
Author: Jack Yu <llacque@gmail.com>
Date:   Wed May 29 10:39:14 2019 +0800

    band09 h2o None cosz=1.0 nf_refs=[4, 6]
    
    atmpro None
    band 9
    commitnumber None
    conc atmpro
    cosz 1.0
    dv 0.001
    klin 3e-25
    molecule h2o
    ng_adju [-3, 0]
    ng_refs [4, 6]
    nv 10000
    option_k_lookup 0
    ref_pts [(300, 250), (300, 250)]
    rsfc 0
    tsfc None
    vmax 14290
    vmin 8200
    wgt [(0.95, 0.9, 0.5, 0.5), (0.5, 0.5, 0.5, 0.7, 0.85, 0.95)]


## Get nbviewer url   
We want to be able to conveniently obtain the url that points to the uploaded jupyter notebook.

In [526]:
PRE_URL = Path('https://nbviewer.jupyter.org/github/qap')

In [535]:
gitname = 'analysis_-_new_kdist_param_sw'
if ('/' + gitname + '/') in str(PATH): _, suf = str(PATH).split(gitname)
else: suf = PATH
suf = 'blob/master' + suf
suf

'blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]'

In [536]:
PRE_URL/gitname/suf/'analysis_-_lblnew-bestfit-sw.ipynb'

PosixPath('https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')

## Putting things together -- LBLnewBestfitSWAnalysis class

In [577]:
#export
class LBLnewBestfitSWAnalysis(object):
    def __init__(self, path, runner):
        self.path = Path(path)
        self.runner = runner
        self.path.mkdir(exist_ok=True, parents=True)
        
    def input_params(self):
        with open('analysis_-_lblnew-bestfit-sw.ipynb') as f:  # Need to set to stored absolute path
            nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)
        nb['cells'][2]['source'] = f'''PARAM = LBLnewBestfitSWParam(**{vars(self.runner.param)})'''
        nb['cells'][3]['source'] = f'''PATH = Path("{self.runner.path}")'''
        return nb
        
    def run(self):
        nb = self.input_params()
        ep = ExecutePreprocessor(timeout=600)
        ep.preprocess(nb, {})
        with open(self.path/'analysis_-_lblnew-bestfit-sw.ipynb', mode='w', encoding='utf-8') as f:
            nbformat.write(nb, f)
            
    def gitcommit(self):
        cwd = os.getcwd()
        os.chdir(self.path)
        try:
            proc = subprocess.Popen(['git', 'add', 'analysis_-_lblnew-bestfit-sw.ipynb'], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            out, err = proc.communicate()
            print('Git-add', out.decode(), err.decode())
            title = f"band{self.runner.param.band:02d} {self.runner.param.molecule} {self.runner.param.atmpro} cosz={self.runner.param.cosz} nf_refs={self.runner.param.ng_refs}"
            body = '\n'.join(sorted(f"{n} {v}" for n, v in vars(self.runner.param).items()))
            proc = subprocess.Popen(f'''git commit -m "{title}" -m "{body}"''', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            return proc
        finally: os.chdir(cwd)

    def nbviewer_url(self, gitname):
        '''
        gitname: str
            Git repository name.
        '''
        pre_url = Path('https://nbviewer.jupyter.org/github/qap')
        spath = str(self.path)
        if gitname in spath: 
            _, suf = spath.split(gitname)
        else: 
            suf = spath
        suf = ('blob/master' + suf).strip('/')
        print(suf)
        return pre_url/gitname/suf/'analysis_-_lblnew-bestfit-sw.ipynb'

In [578]:
analysis = LBLnewBestfitSWAnalysis(path=PATH, runner=runner)

In [504]:
analysis.run()

In [506]:
proc = analysis.gitcommit()

Git-add  


In [507]:
out, err = proc.communicate()

In [508]:
print('Git-commit', out.decode(), err.decode())

Git-commit [master 57b9cc9] band09 h2o saw cosz=1 nf_refs=[3, 9]
 1 file changed, 1833 insertions(+), 1841 deletions(-)
 rewrite atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb (73%)
 


In [580]:
analysis.nbviewer_url('analysis_-_new_kdist_param_sw')

blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]


PosixPath('https:/nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param_sw/blob/master/atmpro_saw/band_9/commitnumber_79f2292/conc_atmpro/cosz_1/dv_0.001/klin_3e-25/molecule_h2o/ng_adju_[-3, 0]/ng_refs_[3, 9]/nv_10000/option_k_lookup_0/ref_pts_[(30, 250), (300, 250)]/rsfc_0/tsfc_None/vmax_14290/vmin_8200/wgt_[(0.95, 0.9, 0.6, 0.5), (0.55, 0.55, 0.55, 0.65, 0.65, 0.7, 0.8, 0.9)]/analysis_-_lblnew-bestfit-sw.ipynb')

# Export notebook

In [581]:
! python notebook2script.py 03_analysis.ipynb

Converted 03_analysis.ipynb to exp/nb_03.py


# fin